# Sqlite with pandas

https://cldellow.com/2018/06/22/sqlite-parquet-vtable.html

## Setup libraries

In [1]:
import pandas as pd
# import modin.pandas as pd  # does not work
import numpy as np
from faker import Faker
import datetime as dt
from pathlib import Path

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

## Create data

In [3]:
fake = Faker()

In [4]:
def create_rows(num=1):
    output = [{
        "name": fake.name(),
        "state": fake.state(),
        "date_time": fake.date_time(end_datetime=dt.datetime(1990, 1, 1)),
        "randomdata": np.random.randint(10, 100)  # two digit number
    } for x in range(num)]
    return output

Read or create data frame

In [5]:
data_file = Path('random_data.parq.gz')

In [6]:
if data_file.exists():
    df = pd.read_parquet(data_file)
else:
    df = pd.DataFrame(create_rows(num=3_000_000))
    df.to_parquet(data_file, index=False, compression='gzip')

In [7]:
df.head()
# data_file = Path('random_data.csv.gz')
# df.to_csv(data_file, index=False, compression='gzip')
# df2 = pd.read_csv(data_file)

,name,state,date_time,randomdata
0,Lisa Tucker,Vermont,1982-09-23 02:13:32,17
1,Jonathan Drake,California,1982-06-28 04:53:28,32
2,Ruth Cohen,Delaware,1980-10-26 10:11:03,90
3,Samuel Stephens,Tennessee,1973-11-20 00:05:53,94
4,Erik Osborne,Nevada,1971-02-07 23:32:13,43


Find non-unique name state combinations

In [8]:
# name_state = df.groupby(['name', 'state']).size()
# name_state_duplicate = name_state[name_state > 1].to_frame('number')
# name_state_duplicate.head()
# combined = df.merge(name_state_duplicate, how='left', on=['name', 'state'])

Merge data

In [9]:
combined = df.merge(df, how='left', on=['name', 'state'])

In [10]:
combined.shape

(5775252, 6)

In [11]:
combined.head()

,name,state,date_time_x,randomdata_x,date_time_y,randomdata_y
0,Lisa Tucker,Vermont,1982-09-23 02:13:32,17,1982-09-23 02:13:32,17
1,Jonathan Drake,California,1982-06-28 04:53:28,32,1982-06-28 04:53:28,32
2,Jonathan Drake,California,1982-06-28 04:53:28,32,1985-08-16 16:52:52,29
3,Ruth Cohen,Delaware,1980-10-26 10:11:03,90,1980-10-26 10:11:03,90
4,Samuel Stephens,Tennessee,1973-11-20 00:05:53,94,1973-11-20 00:05:53,94


## Use sqlite

In [14]:
df.to_sql('data_table', index=False, con=engine, if_exists='replace')

In [15]:
engine.execute('CREATE INDEX join_index ON data_table (name, state)')  # speeds up join

In [16]:
query = '''
    select count(*) number
        -- dt1.name, dt1.state,
        -- dt1.date_time dt1, dt1.randomdata rd1,
        -- dt2.date_time dt2, dt2.randomdata rd2
    from data_table dt1
        left join data_table dt2
            on dt1.name = dt2.name
            and dt1.state = dt2.state
'''

In [17]:
df2 = pd.read_sql_query(query, engine)

In [18]:
df2.head()

,number
0,5775252


In [19]:
engine.dispose()
del engine